<a href="https://colab.research.google.com/github/AokiMasataka/LSTM_sample/blob/master/LSTM_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U torch torchvision

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.6.0+cu101)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.7.0+cu101)


In [ ]:
import torch
from torch import nn
import numpy as np

VOCAB_SIZE = 64
EMBEDDING_DIM = 16
HIDDEN_DIM = 64
sequence = 5

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.word_embeddings = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        self.lstm = nn.LSTM(input_size=EMBEDDING_DIM, hidden_size=HIDDEN_DIM, batch_first=True, num_layers=1)
        self.liner = nn.Linear(HIDDEN_DIM, VOCAB_SIZE)

    def forward(self, sentence, hc=None):
        embeds = self.word_embeddings(sentence)
        _, (hn, cn) = self.lstm(embeds, hc)
        output = self.liner(hn[0])
        return output, (hn, cn)

In [112]:
def getData(batch_size):
  tensor = torch.zeros(sequence).reshape(1, -1)
  for i in range(batch_size):
    numbers = []
    num = np.random.randint(0, VOCAB_SIZE - sequence)
    for i in range(sequence):
      numbers.append(num + i)
    tensor = torch.cat((tensor, torch.tensor(numbers).reshape(1, -1)), 0)
  return tensor[1:].to(torch.long)

In [ ]:
decoder = Decoder()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(decoder.parameters(), lr=0.01, momentum=0.9)

# 学習
decoder.train()
hc = None
for i in range(2000):
  data = getData(4)
  x = data[:, :-1]
  y = data[:, -1]
  output, (h, c) = decoder(x, hc)
  loss = criterion(output, y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if(i + 1) % 100 == 0:
    print("epoch :", loss.item())


In [ ]:
# Test
decoder.eval()
start = [[24]]
test_tensor = torch.tensor(start)

hc = None
for i in range(sequence):
  output, (h, c)= decoder(test_tensor, hc)
  hc = (h, c)
  number = torch.argmax(output).reshape(1, 1)
  test_tensor = torch.cat((test_tensor, number), -1)
  print(test_tensor[0])